**Deep Learning Image Classification using Neural Network**
<br> by 
Matan-Ben Nagar
&
Yaara Kresner-Barak

## Import Necessary Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from keras.layers import Dense
tfds.disable_progress_bar()

## Load rock_paper_scissors dataset
The dataset contains 2892 images of hands playing rock, paper, scissor game.
Its have two features- image (300, 300, 3) and lable.




In [3]:
##import the dataset from tensorflow_datasets library 
builder = tfds.builder('rock_paper_scissors')

Split Rock, Paper, Scissors data
The train set contains 2520 images, and the test set contains 372 images.

In [4]:
#load the train and test sets from the DB 
ds_train = tfds.load(name="rock_paper_scissors", split="train")
ds_test = tfds.load(name="rock_paper_scissors", split="test")

Dataset rock_paper_scissors downloaded and prepared to C:\Users\matan\tensorflow_datasets\rock_paper_scissors\3.0.0. Subsequent calls will reuse this data.


Converting the tensorflow dataset format into numpy format,

Create numpy arrays that contains the images and the labls separately,

And change the images three color channels RGB format to one color channel (to reduce the unimportant data)


In [5]:
train_images = np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example['label'].numpy() for example in ds_train])

test_images = np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example['label'].numpy() for example in ds_test])

Reshaping the images to 300 x 300 x 1 (add color feature- grayscale images). 

In [6]:
train_images = train_images.reshape(2520, 300, 300, 1)
test_images = test_images.reshape(372, 300, 300, 1)

getting us ready to be able to convert it from a scale of 0 to 1
instead of 0 to 255


In [7]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

##Normalize the Images
Train images dividing equal by 255,
So the max value we can have is 255 because RGB values are between 0 and 255 so by doing this we're scaling every value to be between 0 & 1 and this is just a good common practice that helps you classify it.
It helps the basically network learn better than if you use the 0 to 255 values you could leave it 0 to 255 but it's just ultimately it's gonna probably decrease your performance a bit, so it's a common step to normalize between 0 & 1.


In [8]:
train_images /= 255
test_images /= 255

## Logistic Regression

In [15]:
# Output layer.
model_lr = keras.Sequential([
   keras.layers.Flatten(),
   keras.layers.Dense(3, activation='softmax')
])


In [16]:
# adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model_lr.compile(
    optimizer=rmsprop_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [17]:
model_lr.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 1s 8ms/step - loss: 55.8028 - accuracy: 0.3448
Epoch 2/5
79/79 [==============================] - 1s 9ms/step - loss: 38.1533 - accuracy: 0.4274
Epoch 3/5
79/79 [==============================] - 1s 9ms/step - loss: 34.7076 - accuracy: 0.4444
Epoch 4/5
79/79 [==============================] - 1s 9ms/step - loss: 29.2796 - accuracy: 0.5135
Epoch 5/5
79/79 [==============================] - 1s 9ms/step - loss: 27.6304 - accuracy: 0.5163


In [18]:
model_lr.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 7ms/step - loss: 9.2975 - accuracy: 0.5699


[9.297526359558105, 0.5698924660682678]

## Train a basic neural network (first try)

The first model network layer transform the 300 by 300 image into single colum,
After that we have two layers of activation relu function- because the constant gradient of ReLUs results in faster learning.
Finally ,the output layer going to be the same size as the number of labels we trying to classify- we use softmax because it efficient in classification problems. 

In [ ]:
model = keras.Sequential([
  keras.layers.Flatten(),
  Dense(512, activation='relu'),
  Dense(256, activation='relu'),
  Dense(3, activation='softmax')
])

#setup loss function
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

#fit our data to the model
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 5s 35ms/step - loss: 22.9600 - accuracy: 0.4103
Epoch 2/5
79/79 [==============================] - 3s 41ms/step - loss: 1.9036 - accuracy: 0.6675
Epoch 3/5
79/79 [==============================] - 3s 34ms/step - loss: 0.7917 - accuracy: 0.7877
Epoch 4/5
79/79 [==============================] - 3s 35ms/step - loss: 0.6833 - accuracy: 0.7889
Epoch 5/5
79/79 [==============================] - 3s 35ms/step - loss: 0.2984 - accuracy: 0.8944


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 1s 13ms/step - loss: 2.5974 - accuracy: 0.4435


[2.5974009037017822, 0.44354838132858276]

In this case we overfitting to our data - the model not learning the train data. 
we can see it by the results- the accuracy in the train data is 0.89 and tne accu

## Train a Network (convolutional approach)

In [ ]:
model.evaluate(test_images, test_labels)

## A Better Convolutional Network
This time the first layer will be Conv2D()  because the dataset consist of 2D images. The first variable inserted in the function is basically how many times a smaller gird is passing over the image <br>
this is how big or smaller grid is so if I said three and I didn't pass in three to start off and we'll leave the rides at one two one that just means they'll move one every time so it's gonna be a sliding window of three by three 

In [ ]:
model = keras.Sequential([
   keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)),
   keras.layers.Conv2D(64, 3, activation='relu'),
   keras.layers.Conv2D(32, 3, activation='relu'),
   keras.layers.MaxPool2D(2,2),
   keras.layers.Dropout(0.5),
   keras.layers.Flatten(),
   keras.layers.Dense(128, activation='relu'),
   keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images,test_labels)

## Plot Image from Numpy Array

In [ ]:
rgb_images = np.array([example['image'].numpy() for example in ds_train.take(1)])
rgb_image = rgb_images[0]
plt.imshow(rgb_image)
rgb_image.shape

#image = train_images[0].reshape(300,300)
#plt.imshow(image)

# green_images = np.array([example['image'].numpy()[:,:,1] for example in ds_train.take(1)])
# green_image = green_images[0].reshape(300,300)
# plt.imshow(green_image, cmap='Greys_r')

## Use Model to Predict Result for Single Example

In [ ]:
result = best_model.predict(np.array([train_images[0]]))
print(result)

predicted_value = np.argmax(result)
print(predicted_value)

## Convert PNG/JPG images to Numpy Format

In [ ]:
import imageio

im = imageio.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Massachusetts_State_House_-_panoramio_%281%29.jpg/280px-Massachusetts_State_House_-_panoramio_%281%29.jpg')

print(type(im))

im_np = np.asarray(im)

print(im_np.shape)